In [3]:
from audio_diffusion_pytorch.audio_diffusion_pytorch import DiffusionModel, UNetV0, VDiffusion_Inpainted, VSampler
import utils.load_datasets
import utils.training
import utils.logging
from networks import *
import networks.transforms as net_transforms
from torch.utils.data import DataLoader, random_split
import numpy as np
from torchvision import transforms
import torch
from torch.optim import Adam
import matplotlib.pyplot as plt
from pathlib import Path
import os
# Define hyperparameters
train_modulations = ['OOK', '4ASK', '8ASK', 'BPSK', 'QPSK', '8PSK', '16PSK', '32PSK', '16APSK', '32APSK',
                        '64APSK', '128APSK', '16QAM', '32QAM', '64QAM', '128QAM', '256QAM', 'AM-SSB-WC',
                        'AM-SSB-SC', 'AM-DSB-WC', 'AM-DSB-SC', 'FM', 'GMSK', 'OQPSK']
train_SNRs = np.arange(-20, 32, 2)
#test_modulations = ['BPSK', 'QPSK', '8PSK', '16QAM', '64QAM', '256QAM']
test_modulations = ['OOK', '4ASK', '8ASK', 'BPSK', 'QPSK', '8PSK', '16PSK', '32PSK', '16APSK', '32APSK',
                        '64APSK', '128APSK', '16QAM', '32QAM', '64QAM', '128QAM', '256QAM', 'AM-SSB-WC',
                      'AM-SSB-SC', 'AM-DSB-WC', 'AM-DSB-SC', 'FM', 'GMSK', 'OQPSK']
train_modulations = ['AM-SSB', 'CPFSK', 'QPSK', 'GFSK', 'PAM4', 'QAM16', 'WBFM', '8PSK', 'QAM64', 'AM-DSB', 'BPSK']
train_SNRs = np.arange(-20,19,2)
test_SNRs = np.arange(-20, 30, 2)
dataset_train_name = '2016.10A'
dataset_test_name = '2016.10A'
dataDir = '/home/trey/experiment_rfdiffusion/models/saved_models/impainting'
batch_size = 4
learning_rate = 1e-4
adam_betas = (0.9, 0.999)
model_save_dir = '/home/trey/experiment_rfdiffusion/models/saved_models/impainting'
# Create directories if not exist
utils.training.create_directory(dataDir)

# Define data split ratios
split = [0.75, 0.05, 0.20]

# Define data transformations
train_transforms = transforms.Compose([net_transforms.PowerNormalization()])
test_transforms = train_transforms

# Load datasets
train_dataset= utils.load_datasets.getDataset(
    dataset_train_name, dataset_test_name, train_modulations, train_SNRs, test_modulations, test_SNRs, split, dataDir, train_transforms, test_transforms
)

# Create data loader
data_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

# Set device
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

model = DiffusionModel(
    net_t=UNetV0,  # The model type used for diffusion (U-Net V0 in this case)
    in_channels=2,  # U-Net: number of input/output (audio) channels
    channels=[64, 128, 256, 512],  # U-Net: channels at each layer
    factors=[2, 2, 2, 2],  # U-Net: downsampling and upsampling factors at each layer
    items=[2, 2, 2, 2],  # U-Net: number of repeating items at each layer
    attentions=[1, 1, 1, 1],  # U-Net: attention enabled/disabled at each layer
    attention_heads=4,  # U-Net: number of attention heads per attention item
    attention_features=32,  # U-Net: number of attention features per attention item
    diffusion_t=VDiffusion_Inpainted,  # The diffusion method used
    use_text_conditioning=False,  # U-Net: enables text conditioning (default T5-base)
    use_embedding_cfg=False,  # U-Net: enables classifier free guidance
).to(device)

train_modulations = ['AM-SSB', 'CPFSK', 'QPSK', 'GFSK', 'PAM4', 'QAM16', 'WBFM', '8PSK', 'QAM64', 'AM-DSB', 'BPSK']
train_SNRs = np.arange(10,19,2)
num_samples_per_combination = 50  # Number of samples to generate per combination
# Define the path to the checkpoint file
checkpoint_path = '/home/trey/experiment_rfdiffusion/models/saved_models/model_epoch_49.pth'

# Load the checkpoint
checkpoint = torch.load(checkpoint_path)

# Load the state dictionary into the model
model.load_state_dict(checkpoint['model_state_dict'])
dataDir = '/home/trey/experiment_rfdiffusion/models/test/'
utils.training.create_directory(dataDir)
results_folder = Path(dataDir) / 'results'
# If you also saved the optimizer state
optimizer = Adam(model.parameters(), lr=learning_rate, betas=adam_betas)
optimizer.load_state_dict(checkpoint['optimizer_state_dict'])

# Optionally, load the epoch and loss
epoch = checkpoint['epoch']
loss = checkpoint['loss']
print('sampling')
print(f"Checkpoint loaded. Model trained for {epoch} epochs. Last recorded loss: {loss}")
# Turn noise into new audio sample with diffusion
train_modulations = ['BPSK']
train_SNRs = np.arange(16,18,2)
for modulation in train_modulations:
        for snr in train_SNRs: 
            prompt = f"{modulation} modulated waveform at {snr} dB SNR"
            generated_samples = []
            print(prompt)
            for x in range(1, num_samples_per_combination):
                noise = torch.randn(1, 2, 128).to(device)
                sample = model.sample(
                        noise,
                        num_steps=10 # Higher for better quality, suggested num_steps: 10-100
                        )
                sample = sample.cpu().numpy()
                generated_samples.append(sample)
            generated_samples = np.concatenate(generated_samples, axis=0)
            # Generate and save the constellation plot
            plt.figure()
            plt.scatter(generated_samples[:, 0, :].flatten(), generated_samples[:, 1, :].flatten(), marker='.')
            plt.title(f'Constellation Plot - {modulation}, SNR {snr} dB')
            plt.xlabel('I')
            plt.ylabel('Q')
            plt.grid(True)
            filename = results_folder / f'constellation_{modulation}_{snr}dB.png'
            plt.savefig(filename)
            class_index = train_modulations.index(modulation)
            snr_index = np.where(train_SNRs == snr)[0][0]
            # Save the generated samples and labels
            np.savez_compressed(os.path.join(results_folder, f'samples_{modulation}_{snr}dB.npz'), 
                                samples=generated_samples, 
                                classes=np.array([modulation] * num_samples_per_combination), 
                                snrs=np.array([snr] * num_samples_per_combination))
            print('saved')

Directory '/home/trey/experiment_rfdiffusion/models/saved_models/impainting' already exists.


RuntimeError: Error(s) in loading state_dict for DiffusionModel:
	Missing key(s) in state_dict: "net.blocks.2.net.block.blocks.0.block.weight", "net.blocks.2.net.block.blocks.0.block.bias", "net.blocks.2.net.block.blocks.1.block.blocks.0.blocks.0.0.weight", "net.blocks.2.net.block.blocks.1.block.blocks.0.blocks.0.0.bias", "net.blocks.2.net.block.blocks.1.block.blocks.0.blocks.0.2.weight", "net.blocks.2.net.block.blocks.1.block.blocks.0.blocks.0.2.bias", "net.blocks.2.net.block.blocks.1.block.blocks.0.blocks.1.0.weight", "net.blocks.2.net.block.blocks.1.block.blocks.0.blocks.1.0.bias", "net.blocks.2.net.block.blocks.1.block.blocks.0.blocks.1.2.weight", "net.blocks.2.net.block.blocks.1.block.blocks.0.blocks.1.2.bias", "net.blocks.2.net.block.blocks.2.blocks.0.block.blocks.0.1.weight", "net.blocks.2.net.block.blocks.2.blocks.0.block.blocks.0.1.bias", "net.blocks.2.net.block.blocks.3.blocks.0.block.norm.weight", "net.blocks.2.net.block.blocks.3.blocks.0.block.norm.bias", "net.blocks.2.net.